The official Twitter API documentation on the requested endpoints: <br>
*https://developer.twitter.com/en/docs/twitter-api/users/lookup/api-reference/get-users-me* <br>
*https://developer.twitter.com/en/docs/twitter-api/tweets/timelines/api-reference/get-users-id-reverse-chronological*

## 1. Importing Modules

In [ ]:
import requests as r
import os
import json
from dotenv import load_dotenv
import base64
import pandas as pd

In [ ]:
# reading the contents of the .env file stored in the notebook's folder
load_dotenv()

# this setting will make all the 100 records of the dataframe visible when printed
pd.set_option('display.max_rows', 100) 

## 2. Assigning the Request Parameters to Variables

In [ ]:
endpoint_user = 'https://api.twitter.com/2/users/me'

In [ ]:
access_token = os.getenv('access_token_oauth2')
access_token = 'Bearer ' + access_token

In [ ]:
headers = {'Authorization' : access_token}

## 3. The GET Request to Return the User Info

In [ ]:
response_user = r.get(endpoint_user, headers = headers)

if response_user.status_code == 200:
    print('Successfully connected!', 'Status Code:', response_user.status_code)
else:
    print('Something went wrong!', 'Status Code: ', response_user.status_code)

## 4. Converting the Response Output to a Python Object

In [ ]:
# bytes to string
response_json = response_user.content.decode('utf8')

# json string to python object
response_json_formatted = json.loads(response_json)

## 5. Extracting the User Info from the Response

In [ ]:
user_id = response_json_formatted['data']['id']
name = response_json_formatted['data']['name']
username = response_json_formatted['data']['username']

print('User ID:', user_id)
print('Name:', name)
print('Username:', username)

##  6. A GET Request to Import the User's Timeline

In [ ]:
endpoint_timeline = 'https://api.twitter.com/2/users/' + user_id + '/timelines/reverse_chronological'

In [ ]:
# the request parameters

max_results = 100

tweet_fields = 'author_id,context_annotations,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,text,withheld'

user_fields = 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'

place_fields = 'contained_within,country,country_code,full_name,geo,id,name,place_type'

expansions = 'attachments.media_keys,author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id'

params = {
    'max_results' : max_results,
    'tweet.fields' : tweet_fields,
    'user.fields' : user_fields,
    'place.fields' : place_fields,
    'expansions' : expansions
}

In [ ]:
response_timeline = r.get(endpoint_timeline, headers = headers, params = params).json()

## 7. Converting the Output to a Python Object

In [ ]:
# returning the contents of the response
data_json = response_timeline['data']

# converts the contents to a pandas dataframe
data_raw = pd.json_normalize(data_json)

# changing the default index to the 'id' column
data_raw.set_index('id', inplace = True)

##  8. Creating a Copy of the DF to Keep the Original Version Untouched

In [ ]:
data = data_raw.copy()

## 9. Deleting / Renaming Columns

In [ ]:
data.drop(
    [
    'reply_settings',
    'attachments.media_keys', 
    'referenced_tweets', 
    'in_reply_to_user_id', 
    'context_annotations',
    'entities.urls',
    'entities.annotations',
    'entities.mentions',
    'entities.hashtags'
    ], 
    axis = 1,
    inplace = True
)

data.head(1)

In [ ]:
data = data.rename(
    columns = {
        'public_metrics.retweet_count':'retweet_count',
        'public_metrics.reply_count':'reply_count', 
        'public_metrics.like_count':'like_count', 
        'statistics.favoriteCount':'favorite_count', 
        'public_metrics.quote_count':'quote_count'
    }
)

data.head(1)

## 10. Iterating Through the JSON to Extract the Nested URL Column

In [ ]:
entities_expanded_url = []

for i in data_json:
    try:
        x = i['entities']['urls'][0]['display_url']
        entities_expanded_url.append(x)
    except:
        entities_expanded_url.append('NaN')

## 11. Appending the Extracted Into to the Dataframe

In [ ]:
data['url'] = entities_expanded_url

## 12. Checking and Fixing the Data Types

In [ ]:
data.info()

In [ ]:
# changing the 'created_at' column's data type to make it a date column
data['created_at'] = data['created_at'].astype('datetime64[ns]')